# Имплементиране на експертна система за животни

Пример от [Учебна програма по изкуствен интелект за начинаещи](http://github.com/microsoft/ai-for-beginners).

В този пример ще имплементираме проста система, базирана на знания, за определяне на животно въз основа на някои физически характеристики. Системата може да се представи чрез следното AND-OR дърво (това е част от цялото дърво, можем лесно да добавим още правила):

![](../../../../../../translated_images/bg/AND-OR-Tree.5592d2c70187f283.webp)


## Собствена експертна система с обратно извеждане

Нека опитаме да дефинираме прост език за представяне на знания, базиран на продукционни правила. Ще използваме Python класове като ключови думи за дефиниране на правила. Съществено ще има 3 вида класове:
* `Ask` представлява въпрос, който трябва да бъде зададен на потребителя. Той съдържа набора от възможни отговори.
* `If` представлява правило и е просто синтактичен захар за съхраняване на съдържанието на правилото
* `AND`/`OR` са класове за представяне на AND/OR клонове на дървото. Те просто съхраняват списъка с аргументи вътре. За опростяване на кода, цялата функционалност е дефинирана в родителския клас `Content`


In [1]:
class Ask():
    def __init__(self,choices=['y','n']):
        self.choices = choices
    def ask(self):
        if max([len(x) for x in self.choices])>1:
            for i,x in enumerate(self.choices):
                print("{0}. {1}".format(i,x),flush=True)
            x = int(input())
            return self.choices[x]
        else:
            print("/".join(self.choices),flush=True)
            return input()

class Content():
    def __init__(self,x):
        self.x=x
        
class If(Content):
    pass

class AND(Content):
    pass

class OR(Content):
    pass

В нашата система, работната памет ще съдържа списъка с **факти** като **двойки атрибут-стойност**. Базата от знания може да се определи като един голям речник, който свързва действията (нови факти, които трябва да се вмъкнат в работната памет) с условия, изразени като AND-OR изрази. Също така, някои факти могат да бъдат `Ask`.


In [2]:
rules = {
    'default': Ask(['y','n']),
    'color' : Ask(['red-brown','black and white','other']),
    'pattern' : Ask(['dark stripes','dark spots']),
    'mammal': If(OR(['hair','gives milk'])),
    'carnivor': If(OR([AND(['sharp teeth','claws','forward-looking eyes']),'eats meat'])),
    'ungulate': If(['mammal',OR(['has hooves','chews cud'])]),
    'bird': If(OR(['feathers',AND(['flies','lies eggs'])])),
    'animal:monkey' : If(['mammal','carnivor','color:red-brown','pattern:dark spots']),
    'animal:tiger' : If(['mammal','carnivor','color:red-brown','pattern:dark stripes']),
    'animal:giraffe' : If(['ungulate','long neck','long legs','pattern:dark spots']),
    'animal:zebra' : If(['ungulate','pattern:dark stripes']),
    'animal:ostrich' : If(['bird','long nech','color:black and white','cannot fly']),
    'animal:pinguin' : If(['bird','swims','color:black and white','cannot fly']),
    'animal:albatross' : If(['bird','flies well'])
}

За да извършим обратно извод, ще дефинираме клас `Knowledgebase`. Той ще съдържа:
* Работна `memory` - речник, който свързва атрибути със стойности
* Правила на базата знания `rules` във формата, дефиниран по-горе

Два основни метода са:
* `get` за получаване на стойността на атрибут, като при необходимост извършва извод. Например, `get('color')` ще получи стойността на слот за цвят (ще попита ако е необходимо и ще запази стойността за бъдеща употреба в работната памет). Ако поискамe `get('color:blue')`, ще попита за цвят и след това ще върне стойност `y`/`n` в зависимост от цвета.
* `eval` извършва реалния извод, т.е. обхожда AND/OR дърво, оценява подцели и т.н.


In [3]:
class KnowledgeBase():
    def __init__(self,rules):
        self.rules = rules
        self.memory = {}
        
    def get(self,name):
        if ':' in name:
            k,v = name.split(':')
            vv = self.get(k)
            return 'y' if v==vv else 'n'
        if name in self.memory.keys():
            return self.memory[name]
        for fld in self.rules.keys():
            if fld==name or fld.startswith(name+":"):
                # print(" + proving {}".format(fld))
                value = 'y' if fld==name else fld.split(':')[1]
                res = self.eval(self.rules[fld],field=name)
                if res!='y' and res!='n' and value=='y':
                    self.memory[name] = res
                    return res
                if res=='y':
                    self.memory[name] = value
                    return value
        # field is not found, using default
        res = self.eval(self.rules['default'],field=name)
        self.memory[name]=res
        return res
                
    def eval(self,expr,field=None):
        # print(" + eval {}".format(expr))
        if isinstance(expr,Ask):
            print(field)
            return expr.ask()
        elif isinstance(expr,If):
            return self.eval(expr.x)
        elif isinstance(expr,AND) or isinstance(expr,list):
            expr = expr.x if isinstance(expr,AND) else expr
            for x in expr:
                if self.eval(x)=='n':
                    return 'n'
            return 'y'
        elif isinstance(expr,OR):
            for x in expr.x:
                if self.eval(x)=='y':
                    return 'y'
            return 'n'
        elif isinstance(expr,str):
            return self.get(expr)
        else:
            print("Unknown expr: {}".format(expr))

Сега нека дефинираме нашата животинска база знания и да проведем консултацията. Обърнете внимание, че този разговор ще ви задава въпроси. Можете да отговаряте, като въвеждате `y`/`n` за въпроси с отговор да/не, или като посочите число (0..N) за въпроси с по-дълги множествени отговори.


In [4]:
kb = KnowledgeBase(rules)
kb.get('animal')

hair
y/n
sharp teeth
y/n
claws
y/n
forward-looking eyes
y/n
color
0. red-brown
1. black and white
2. other
has hooves
y/n
long neck
y/n
long legs
y/n
pattern
0. dark stripes
1. dark spots


'giraffe'

## Използване на Experta за напредващо извеждане

В следващия пример ще се опитаме да реализираме напредващо извеждане, използвайки една от библиотеките за представяне на знания, [Experta](https://github.com/nilp0inter/experta). **Experta** е библиотека за създаване на системи за напредващо извеждане на Python, която е проектирана да бъде подобна на класическата стара система [CLIPS](http://www.clipsrules.net/index.html).

Можехме също така да реализираме напредващо извеждане сами без много проблеми, но наивните реализации обикновено не са много ефективни. За по-ефективно съвпадение на правилата се използва специален алгоритъм [Rete](https://en.wikipedia.org/wiki/Rete_algorithm).


In [5]:
import sys
!{sys.executable} -m pip install git+https://github.com/nilp0inter/experta

  Cloning https://github.com/nilp0inter/experta to /tmp/pip-req-build-7qurtwk3
  Running command git clone --filter=blob:none --quiet https://github.com/nilp0inter/experta /tmp/pip-req-build-7qurtwk3
  Resolved https://github.com/nilp0inter/experta to commit c6d5834b123861f5ae09e7d07027dc98bec58741
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for experta: filename=experta-1.9.5.dev1-py3-none-any.whl size=34804 sha256=888c459512a5e713f4b674caa9a0f96cfdf07ec0d6eb56cc318ce0653d218014
  Stored in directory: /tmp/pip-ephem-wheel-cache-1eeii9zy/wheels/3d/e8/bb/22d7956359603fa8dd679aa09f5b8efb3f29991c3986fdc787
Successfully built experta
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [experta]


In [13]:
from experta import *
#import experta

Ще дефинираме нашата система като клас, който наследява `KnowledgeEngine`. Всяко правило се дефинира чрез отделна функция с анотация `@Rule`, която указва кога правилото трябва да се изпълни. Вътре в правилото можем да добавим нови факти, използвайки функцията `declare`, и добавянето на тези факти ще доведе до извикване на още правила от напредналия инференчен механизъм.


In [14]:
class Animals(KnowledgeEngine):
    @Rule(OR(
           AND(Fact('sharp teeth'),Fact('claws'),Fact('forward looking eyes')),
           Fact('eats meat')))
    def cornivor(self):
        self.declare(Fact('carnivor'))
        
    @Rule(OR(Fact('hair'),Fact('gives milk')))
    def mammal(self):
        self.declare(Fact('mammal'))

    @Rule(Fact('mammal'),
          OR(Fact('has hooves'),Fact('chews cud')))
    def hooves(self):
        self.declare('ungulate')
        
    @Rule(OR(Fact('feathers'),AND(Fact('flies'),Fact('lays eggs'))))
    def bird(self):
        self.declare('bird')
        
    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark spots'))
    def monkey(self):
        self.declare(Fact(animal='monkey'))

    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark stripes'))
    def tiger(self):
        self.declare(Fact(animal='tiger'))

    @Rule(Fact('ungulate'),
          Fact('long neck'),
          Fact('long legs'),
          Fact(pattern='dark spots'))
    def giraffe(self):
        self.declare(Fact(animal='giraffe'))

    @Rule(Fact('ungulate'),
          Fact(pattern='dark stripes'))
    def zebra(self):
        self.declare(Fact(animal='zebra'))

    @Rule(Fact('bird'),
          Fact('long neck'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def straus(self):
        self.declare(Fact(animal='ostrich'))

    @Rule(Fact('bird'),
          Fact('swims'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def pinguin(self):
        self.declare(Fact(animal='pinguin'))

    @Rule(Fact('bird'),
          Fact('flies well'))
    def albatros(self):
        self.declare(Fact(animal='albatross'))
        
    @Rule(Fact(animal=MATCH.a))
    def print_result(self,a):
          print('Animal is {}'.format(a))
                    
    def factz(self,l):
        for x in l:
            self.declare(x)

След като дефинираме база знания, попълваме работната си памет с някои първоначални факти и след това извикваме метода `run()`, за да извършим извод. Като резултат можете да видите, че нови изведени факти се добавят към работната памет, включително и крайния факт за животното (ако сме задали всички първоначални факти правилно).


In [15]:
ex1 = Animals()
ex1.reset()
ex1.factz([
    Fact(color='red-brown'),
    Fact(pattern='dark stripes'),
    Fact('sharp teeth'),
    Fact('claws'),
    Fact('forward looking eyes'),
    Fact('gives milk')])
ex1.run()
ex1.facts

Animal is tiger


FactList([(0, InitialFact()),
          (1, Fact(color='red-brown')),
          (2, Fact(pattern='dark stripes')),
          (3, Fact('sharp teeth')),
          (4, Fact('claws')),
          (5, Fact('forward looking eyes')),
          (6, Fact('gives milk')),
          (7, Fact('mammal')),
          (8, Fact('carnivor')),
          (9, Fact(animal='tiger'))])

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Отказ от отговорност**:
Този документ е преведен чрез AI преводаческа услуга [Co-op Translator](https://github.com/Azure/co-op-translator). Въпреки че се стремим към точност, моля, имайте предвид, че автоматизираните преводи могат да съдържат грешки или неточности. Оригиналният документ на първичния му език трябва да се счита за авторитетен източник. За критична информация се препоръчва професионален човешки превод. Ние не носим отговорност за каквито и да е неразбирателства или неправилни тълкувания, произтичащи от използването на този превод.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
